MachineHack Subscriber Prediction Challenge


https://machinehack.com/hackathons/subscriber_prediction_talent_search_hackathon/data

In [ ]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
import catboost as cb
import optuna
from sklearn.metrics import log_loss, roc_auc_score

In [ ]:
train= pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sub = pd.read_csv('data/submission.csv')

In [ ]:
train.head()

In [ ]:
cat_cols = train.select_dtypes(include=['object']).columns.to_list()

In [ ]:
cat_cols

In [ ]:
train['job'].unique()

In [ ]:
clf = cb.CatBoostClassifier(cat_features=cat_cols)

In [ ]:
train.y_bool.value_counts()

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.drop(['y_bool'], axis=1), train.y_bool, test_size=0.2, random_state=42,stratify=train.y_bool)

In [ ]:
clf = cb.CatBoostClassifier(cat_features=cat_cols,od_type='Iter',od_wait=10)

In [ ]:
clf.fit(xtrain, ytrain, eval_set=(xvalid, yvalid), use_best_model=True, verbose=False,plot=True)

In [ ]:
def objective(trial):
    #catboost optuna for hyperparameter tuning
    param = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 2, 7),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 2, 30),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.1, 1.0),
        'eval_metric': 'Logloss',
        'use_best_model': True,
        'random_seed': 42,
        'logging_level': 'Silent',
        # 'task_type': 'GPU',
        # 'cat_features': cat_cols,
        # 'early_stopping_rounds': 30,
        'od_type':'Iter',
        'od_wait':20,
        
        
    }
    if param["bootstrap_type"] == "Bayesian":
            param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)
    
    model = cb.CatBoostClassifier(**param, cat_features=cat_cols)
    model.fit(xtrain, ytrain, eval_set=(xvalid, yvalid), verbose=0)
    ll = log_loss(yvalid,model.predict_proba(xvalid)[:,-1])
    return ll

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1000)# , timeout=600)

In [ ]:
pred = clf.predict_proba(test)[:,1]

In [ ]:
sub.columns

In [ ]:
sub['y_bool'] = pred

In [ ]:
sub.to_csv("sub.csv", index=False)